## [Python实战异步爬虫(协程)+分布式爬虫(多进程)](https://blog.csdn.net/SL_World/article/details/86633611)

在讲解之前，我们先来通过一幅图看清多进程和协程的爬虫之间的原理及其区别。(图片来源于网络)

![aiohttp](aiohttp.png)

这里，异步爬虫不同于多进程爬虫，它使用单线程(即仅创建一个事件循环，然后把所有任务添加到事件循环中)就能并发处理多任务。在轮询到某个任务后，当遇到耗时操作(如请求URL)时，挂起该任务并进行下一个任务，当之前被挂起的任务更新了状态(如获得了网页响应)，则被唤醒，程序继续从上次挂起的地方运行下去。极大的减少了中间不必要的等待时间。

对于协程(Asyncio库)的原理及实现请见：[Python异步IO之协程(详解)](https://blog.csdn.net/SL_World/article/details/86597738)
对于多进程的知识讲解及实现请见：[廖雪峰-Python多进程](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000)
在有了Asyncio异步IO库实现协程后，我们还需要实现异步网页请求。因此，aiohttp库应运而生。

### 使用aiohttp库实现异步网页请求

在我们写普通的爬虫程序时，经常会用到requests库用以请求网页并获得服务器响应。而在协程中，由于requests库提供的相关方法不是可等待对象(awaitable),使得无法放在await后面，因此无法使用requests库在协程程序中实现请求。

在此，官方专门提供了一个aiohttp库，用来实现异步网页请求等功能，简直就是异步版的requests库，当然需要我们手动安装该库

【基础实现】：在官方文档中，推荐使用ClientSession()函数来调用网页请求等相关方法。

- 首先，我们需要引入aiohttp模块。
- 然后，我们在协程中使用ClientSession()的get()或request()方法来请求网页。(其中async with是异步上下文管理器，其封装了异步实现等功能)

In [ ]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get('http://httpbin.org/get') as resp:
        print(resp.status)
        print(await resp.text())

###  测试普通爬虫程序

下面是一个普通的同步代码，实现顺序爬取10个URL的title。

In [6]:
import time
from lxml import etree
import requests

urls = [
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16488',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16583',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16380',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16911',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16581',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16674',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16112',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/17343',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16659',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16449',
]

def get_title(url, cnt):
    response = requests.get(url)
    html = response.content
    title = etree.HTML(html).xpath('//*[@id="title"]/text()')
    print('第 {} 个 title: {}'.format(cnt,''.join(title)))
    
if __name__ == '__main__':
    start = time.time()
    i = 0
    for url in urls:
        i += 1
        start_each = time.time()
        get_title(url,i)
        print('第 {} 个title 爬取耗时：{}'.format(i,float(time.time()-start_each)))
    print('爬取总耗时：{}秒'.format(float(time.time()-start)))

第 1 个 title: Norm Conflict Resolution in Stochastic Domains
第 1 个title 爬取耗时：1.2110004425048828
第 2 个 title: Algorithms for Trip-Vehicle Assignment in Ride-Sharing
第 2 个title 爬取耗时：1.9450290203094482
第 3 个 title: Tensorized Projection for High-Dimensional Binary Embedding
第 3 个title 爬取耗时：2.9749748706817627
第 4 个 title: Synthesis of Programs from Multimodal Datasets
第 4 个title 爬取耗时：1.2339951992034912
第 5 个 title: Video Summarization via Semantic Attended Networks
第 5 个title 爬取耗时：1.491034746170044
第 6 个 title: TIMERS: Error-Bounded SVD Restart on Dynamic Networks
第 6 个title 爬取耗时：2.0010170936584473
第 7 个 title: Memory Management With Explicit Time in Resource-Bounded Agents
第 7 个title 爬取耗时：3.511124849319458
第 8 个 title: Mitigating Overexposure in Viral Marketing
第 8 个title 爬取耗时：1.5009915828704834
第 9 个 title: Neural Link Prediction over Aligned Networks
第 9 个title 爬取耗时：2.6780362129211426
第 10 个 title: Dual Deep Neural Networks Cross-Modal Hashing
第 10 个title 爬取耗时：5.672969102859497
爬取总耗时：24.

可见，平均每请求完一个URL并解析该HTML耗时2.6秒左右。本次程序运行总耗时24.3秒。

### 测试基于协程的异步爬虫程序

下面，是使用了协程的异步爬虫程序。etree模块用于解析HTML，aiohttp是一个利用asyncio的库，它的API看起来很像请求的API，可以暂时看成协程版的requests。

In [ ]:
import time
from lxml import etree
import aiohttp,asyncio

urls = [
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16488',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16583',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16380',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16911',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16581',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16674',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16112',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/17343',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16659',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16449',
]

titles = []
sem = asyncio.semaphore(10)  # 信号量，控制协程数，防止爬的过快

async def get_title(url):
    


def get_title(url, cnt):
    response = requests.get(url)
    html = response.content
    title = etree.HTML(html).xpath('//*[@id="title"]/text()')
    print('第 {} 个 title: {}'.format(cnt,''.join(title)))
    
if __name__ == '__main__':
    start = time.time()
    i = 0
    for url in urls:
        i += 1
        start_each = time.time()
        get_title(url,i)
        print('第 {} 个title 爬取耗时：{}'.format(i,float(time.time()-start_each)))
    print('爬取总耗时：{}秒'.format(float(time.time()-start)))

## [Python-aiohttp百万并发(上)](https://www.aikaiyuan.com/10935.html)

本文将测试python aiohttp的极限，同时测试其性能表现，以分钟发起请求数作为指标。大家都知道，当应用到网络操作时，异步的代码表现更优秀，但是验证这个事情，同时搞明白异步到底有多大的优势以及为什么会有这样的优势仍然是一件有趣的事情。为了验证，我将发起1000000请求，用aiohttp客户端。aiohttp每分钟能够发起多少请求？你能预料到哪些异常情况以及崩溃会发生，当你用比较粗糙的脚本去发起如此大量的请求？面对如此大量的请求，哪些主要的陷阱是你需要去思考的？

### 初识 asyncio/aiohttp

异步编程并不简单。相比平常的同步编程，你需要付出更多的努力在使用回调函数，以事件以及事件处理器的模式进行思考。同时也是因为asyncio相对较新，相关的教程以及博客还很少的缘故。官方文档非常简陋，只有最基本的范例。在我写本文的时候，Stack Overflow上面，只有410个与asyncio相关的话题（相比之下，twisted相关的有2585）。有个别关于asyncio的不错的博客以及文章，比如这个、这个、这个，或者还有这个以及这个。

简单起见，我们先从基础开始 —— 简单HTTP hello world —— 发起GET请求，同时获取一个单独的HTTP响应。

同步模式，你这么做：

In [9]:
import requests 
def hello():
     return requests.get("http://httpbin.org/get")     
print(hello())

<Response [200]>


接着我们使用aiohttp：

In [2]:
import asyncio
from aiohttp import ClientSession

async def hello():
    async with ClientSession() as session:
        async with session.get("http://httpbin.org/get") as response:
            response = await response.read()
            print(response)
            
loop = asyncio.get_event_loop()
loop.run_until_complete(hello())

b'{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }, \n  "origin": "183.14.132.166, 183.14.132.166", \n  "url": "https://httpbin.org/get"\n}\n'


你使用async以及await关键字将函数异步化。在hello()中实际上有两个异步操作：首先异步获取相应，然后异步读取响应的内容。

Aiohttp推荐使用ClientSession作为主要的接口发起请求。ClientSession允许在多个请求之间保存cookie以及相关对象信息。Session(会话)在使用完毕之后需要关闭，关闭Session是另一个异步操作，所以每次你都需要使用async with关键字。

一旦你建立了客户端session，你可以用它发起请求。这里是又一个异步操作的开始。上下文管理器的with语句可以保证在处理session的时候，总是能正确的关闭它。

要让你的程序正常的跑起来，你需要将他们加入事件循环中。所以你需要创建一个asyncio loop的实例， 然后将任务加入其中。

看起来有些困难，但是只要你花点时间进行思考与理解，就会有所体会，其实并没有那么复杂。

### 访问多个链接

现在我们来做些更有意思的事情，顺序访问多个链接。

同步方式如下：

```
for url in urls:     
    print(requests.get(url).text)
```

很简单。不过异步方式却没有这么容易。所以任何时候你都需要思考，你的处境是否有必要用到异步。如果你的app在同步模式工作的很好，也许你并不需要将之迁移到异步方式。如果你确实需要异步方式，这里会给你一些启示。我们的异步函数hello()还是保持原样，不过我们需要将之包装在asyncio的Future对象中，然后将Future对象列表作为任务传递给事件循环。

In [4]:
import asyncio
from aiohttp import ClientSession

async def hello(url):
    async with ClientSession() as session:
        async with session.get(url) as response:
            response = await response.read()
            print(response)
            
loop = asyncio.get_event_loop()
tasks = []
url = 'http://httpbin.org/headers'
for i in range(5):
    task = asyncio.ensure_future(hello(url))
    tasks.append(task)
    
loop.run_until_complete(asyncio.wait(tasks))

b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'


({<Task finished coro=<hello() done, defined at <ipython-input-4-a2db41345e25>:4> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-4-a2db41345e25>:4> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-4-a2db41345e25>:4> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-4-a2db41345e25>:4> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-4-a2db41345e25>:4> result=None>},
 set())

现在假设我们想获取所有的响应，并将他们保存在同一个列表中。目前，我们没有保存响应内容，仅仅只是打印了他们。让我们返回他们，将之存储在一个列表当中，最后再打印出来。

为了达到这个目的，我们需要修改一下代码：

In [1]:
import asyncio
from aiohttp import ClientSession

async def hello(url):
    async with ClientSession() as session:
        async with session.get(url) as response:
            response = await response.read()
            print(response)
            
async def run(r):
    url = 'http://httpbin.org/headers'
    tasks = []
    for i in range(r):
        task = asyncio.ensure_future(hello(url))
        tasks.append(task)
        responses = await asyncio.gather(*tasks)
        print(responses)
        
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(4)) 
loop.run_until_complete(future)

b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
[None]
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
[None, None]
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
[None, None, None]
b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'
[None, None, None, None]


注意asyncio.gather()的用法，它搜集所有的Future对象，然后等待他们返回。

### 常见错误

现在我们来模拟真实场景，去调试一些错误，作为演示范例。

看看这个：

In [11]:
async def hello(url):
    async with ClientSession() as session:
        async with session.get(url) as response:
            return response.read()
        
url = 'http://httpbin.org/headers'        
loop = asyncio.get_event_loop() 
loop.run_until_complete(hello(url))

<coroutine object ClientResponse.read at 0x05105450>

发生了什么？你期待获得响应对象，但是你得到的是一组协程。怎么会这样？

response.read()是一个异步操作，这意味着它不会立即返回结果，仅仅返回协程。协程需要被调用跟运行，但是这并不是默认行为。在Python34中加入的yield from以及Python35中加入的await便是为此而生。它们将迭代这些协程。以上代码只需要在response.read()前加上await关键字即可修复。如下：

In [12]:
async def hello(url):
    async with ClientSession() as session:
        async with session.get(url) as response:
            response = await response.read()
            print(response)
            
url = 'http://httpbin.org/headers'        
loop = asyncio.get_event_loop() 
loop.run_until_complete(hello(url))

b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.4.4"\n  }\n}\n'


## [Python aiohttp异步爬虫(萌新读物，大神勿扰)](https://www.jianshu.com/p/63d9741b0bee)

## [异步网络模块之aiohttp的使用(一)](https://www.cnblogs.com/c-x-a/p/9248906.html)

平时我们也许用的更多的是requests模块，或者是requests_hml模块，但是他们都属于阻塞类型的不支持异步，速度很难提高，于是后来出现了异步的grequests,开始了异步网络请求，速度得到了大大的提升，但是今天我们要说的另外的一个比较异步网络请求模块-aiohttp。

### 什么是aiohhtp?

要学习一个模块，首先要知道这个模块都能做什么，我们从官网上找到了关于aiohttp的解释。

- Asynchronous HTTP Client/Server for asyncio and Python.
- Supports both Client and HTTP Server.
- Supports both Server WebSockets and Client WebSockets out-of-the-box without the Callback Hell.
- Web-server has Middlewares, Signals and pluggable routing.

大概意思是说aiohttp是一个异步http网络模块分为了客户端和服务端，同时支持websocket的使用，另外不用担心在使用过程中多次回调导致回调地狱情况出现。

### pip安装

`pip3 install aiohttp`

另外官方还推荐安装如下两个库，一个是字符编码检测库 cchardet，另一个是加速 DNS 解析库 aiodns，安装命令如下：

`pip3 install cchardet aiodns`

### 如何使用

我们先看一段代码，后面会对代码作出解释。

In [6]:
import asyncio,aiohttp
from lxml import html

headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}

async def get_source(url):
    conn = aiohttp.TCPConnector(ssl=False) # 防止ssl报错
    async with aiohttp.ClientSession(connector=conn) as session: # 创建session
        async with session.get(url, headers=headers, timeout =60) as response:  #获得请求
            if response.status == 200:  #判断请求码
                source = await response.text()  #使用await关键字获取返回结果
                print(html.fromstring(source).xpath('//title/text()')[0])  #获取网页标题
            else:
                print('failed to visit')
                
if __name__ == '__main__':
    full_urllist = ["https://www.baidu.com","https://www.cnblogs.com","https://www.jianshu.com"]
    event_loop = asyncio.get_event_loop()
    tasks = [get_source(url) for url in full_urllist] #创建事件循环
    results = event_loop.run_until_complete(asyncio.wait(tasks))  #等待任务结束

百度一下，你就知道
简书 - 创作你的创作
博客园 - 代码改变世界


首先定义了连接器并取消ssl安全验证，我们使用verify_ssl使其等于False，默认是True的。

#### TCPConnector

因为有的网站请求的时候会验证ssl证书,如果是自签名的ssl证书会出错。

`conn=aiohttp.TCPConnector(verify_ssl=False)#防止ssl报错`

下面是关于TCPConnector的所有参数部分。

> class aiohttp.TCPConnector(*, ssl=None, verify_ssl=True, fingerprint=None, use_dns_cache=True, ttl_dns_cache=10, family=0, ssl_context=None, local_addr=None, resolver=None, keepalive_timeout=sentinel, force_close=False, limit=100, limit_per_host=0, enable_cleanup_closed=False, loop=None)

**limit** 为了限制同时打开的连接数量，我们可以将限制参数传递给连接器：

`conn = aiohttp.TCPConnector(limit=30)`
同时最大进行连接的连接数为30，默认是100，limit=0的时候是无限制

**limit_per_host**

`conn = aiohttp.TCPConnector(limit_per_host=30) #默认是0`
同一端点的最大连接数量。同一端点即(host, port, is_ssl)完全相同.

#### ClientSession

首先我们创建一个session对象，向下面这样使用async声明异步,同时with上下文关键字,省去了关闭连接的代码，

`async with aiohttp.ClientSession(connector=conn) as session: `

下面是ClientSession的所有参数，这里用的比较多的是connector,headers,cookies其他的参数大家可以去自己探索一下。

> class aiohttp.ClientSession(*, connector=None, loop=None, cookies=None, headers=None, skip_auto_headers=None, auth=None, json_serialize=json.dumps, version=aiohttp.HttpVersion11, cookie_jar=None, read_timeout=None, conn_timeout=None, timeout=sentinel, raise_for_status=False, connector_owner=True, auto_decompress=True, proxies=None

#### session.get

上面我们创建了session对象，然后我们就要进行请求具体的网站了。

`async with session.get(url,headers=headers,timeout=60) as response:`

这一步我们像使用requests那样传入headers参数并指定最大超时为60s。

#### ClientResponse

然后我们判断请求是否情况，之后我们使用 `await response.text()` 获取了网页的源码，注意这里必须使用await关键字来获取协程的结果。然后我们使用了lxml模块获取这三个网页的title标题。

#### 运行

到目前为止我们只是定义了一个协程，并没有真正的运行它接下来我们看看main方法是做什么的。

```
full_urllist= ["https://www.baidu.com","https://www.cnblogs.com","https://www.jianshu.com"]
event_loop = asyncio.get_event_loop() #创建时间循环
tasks = [getsource(url) for url in full_urllist]
results = event_loop.run_until_complete(asyncio.wait(tasks))#等待任务结束
```
         
首先我定义了一个列表含有三个目标url，当前你可以定义更多。

- asyncio.get_event_loop方法可以创建一个事件循环
- 然后使用run_until_complete将协程注册到事件循环，并启动事件循环。
- 协程对象不能直接运行，在注册事件循环的时候，其实是run_until_complete方法将协程包装成为了一个任务（task）对象。所谓task对象是Future类的子类。保存了协程运行后的状态，用于未来获取协程的结果。

> asyncio.ensure_future(coroutine) 和 loop.create_task(coroutine)都可以创建一个task，run_until_complete的参数是一个futrue对象。当传入一个协程，其内部会自动封装成task，task是Future的子类。isinstance(task, asyncio.Future)将会输出True。